In [2]:
import wandb

api = wandb.Api()
runs = api.runs("eth-ticinesi/FORL_Briscola")

In [3]:
import pandas as pd
import os
os.rmdir('./nocom')
com_models = []
nocom_models = []
for run in runs:
    if 'ELO_POOL' not in run.tags or 'COM' in run.tags :
        continue

    comm = 'nocom'
    arch = 'nn' if 'NN' in run.tags else 'rnn'
    num = len(run.logged_artifacts())
    for artifact in run.logged_artifacts():
        if artifact.type != 'model':
            continue
        artifact.download('./tmp/')
        original_name = f"{artifact.name.split(':')[0]}.pth"

        last = False
        if 'policy_train' in artifact.name:
            step = artifact.name.split('policy_train_')[1].split(':')[0]
        elif 'policy_last' in artifact.name:
            step = artifact.name.split('policy_last_')[1].split(':')[0]
            last = True
            
        path = f'{run.id}_{step}.pt'
        os.rename(f'./tmp/{original_name}', f'./{comm}/{path}')
        hidden_dim = run.config['hidden_dim'] if 'hidden_dim' in run.config else 64
        rnn_out = run.config['rnn_out_size'] if 'rnn_out_size' in run.config else 0
        run_pd = pd.DataFrame({'name' : run.name, 'id': run.id, 'arch': arch, 'comm': comm, 'hidden_dim': hidden_dim, 'rnn_out_size': rnn_out, 'step' : step, 'last': last, 'file_name': path }, index=[0])
        nocom_models.append(run_pd)

OSError: [Errno 39] Directory not empty: './nocom'

In [ ]:
pd.concat(nocom_models).to_csv('nocom_models.csv')